In [1]:
import pandas as pd
import pickle
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
import datetime

import speech_recognition as sr
import pyttsx3 
import pyaudio

In [2]:
# Reading csv file

data = pd.read_csv("Chatbot Data.csv", encoding = "ISO-8859-1", engine='python')

data.head()

,Question,Answer,Category
0,What is the definition of Basic Savings Bank ...,The Basic Savings Bank Deposit Account or BSBD...,Basic Savings Bank Deposit Account
1,Whether the guidelines issued on no-frills acc...,No. In supersession of instructions contained ...,Basic Savings Bank Deposit Account
2,Can an Individual have any number of 'Basic Sa...,No. An individual is eligible to have only one...,Basic Savings Bank Deposit Account
3,Whether a 'Basic Savings Bank Deposit Account'...,Holders of 'Basic Savings Bank Deposit Account...,Basic Savings Bank Deposit Account
4,Can an individual have other deposit accounts ...,"Yes. One can have Term/Fixed Deposit, Recurrin...",Basic Savings Bank Deposit Account


In [3]:
data.columns = ['Question','Answer','Class']

In [4]:
#defining stopwords

stop_words = set(stopwords.words('english'))

In [5]:
# Defining label encoder and TfidfVectorizer

le = LE()

tfv = TfidfVectorizer(min_df=1, stop_words='english')

In [6]:
questions = data['Question'].values

In [7]:
#Tokenization and stopword removal

def cleanup(sentence):




    word_tok = nltk.word_tokenize(sentence)
    stemmed_words = [w for w in word_tok if not w in stop_words]
    #print(stemmed_words)

    #stemmed_words = [stemmer.stem(w) for w in word_tok]



    return ' '.join(stemmed_words)

In [8]:
X = []

for question in questions:
    X.append(cleanup(str(question)))

In [9]:
#vectorization

#tfv.fit(X)

from sentence_transformers import SentenceTransformer

t_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

le.fit(data['Class'])

X = t_model.encode(X)

y = le.transform(data['Class'])


In [10]:
t_model.save("Tranformer_saved")

In [11]:
model = SVC(kernel='linear')
model.fit(X, y)

SVC(kernel='linear')

In [12]:
#dumping the model

with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [13]:

# #loading the model

with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)

In [14]:
svm_predicted_values = model.predict(X)

In [15]:
from sklearn.metrics import accuracy_score

print("SVM : ",accuracy_score(y, svm_predicted_values))

SVM :  0.9896373056994818
